# Tarea 1B
***Autores:** Pablo Montes, Joaquín Pelle y Oscar Reula*
### Importante 1: ###
Este notebook está diseñado para ejecutar en Julia.

### Importante 2: ###
Renombre el archivo como (según corresponda): 
**apellido_nombre_tarea_1.ipynb** o **grupo_tarea_1.ipynb**

**Todo el código generado y la presentación deberá estar incluida en este notebook.**

**Antes de entregar, serciorese que todas las variables estén bien definidas, para ello interrumpa el kernel y corra todo desde cero.**

**El objetivo de las primeras tareas (A y B) es variado:**

1. Construir algunos integradores simples y aplicarlos a problemas en 1 y 2 dimensiones.
2. Estudiar convergencia y determinar el factor Q del método.
3. Estudia la región de convergencia de los métodos.
4. Comprobar cualitativa y cuantitativamente los errores de la aproximación.
5. Familiarizarse con las librerías de integradores de ODEs.
6. Familiarizarse con el manejo y estudio de soluciones para casos de complejidad media.
7. Estudiar el caso stiff y los métodos IMEX.

### Nota importante
A partir de ahora utilizaremos el paquete `DifferentialEquations`, que tiene ya implementados los integradores de Euler, Runge-Kutta, y otros. Haga todos los ejercicios utilizando este paquete.

In [1]:
import Pkg; Pkg.add("DifferentialEquations")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
#=
using Pkg
Pkg.add("DifferentialEquations")
=#

using DifferentialEquations

## Problema 1
En el directorio `Dipolo_mag_3d` encontrará un notebook de Julia que realiza una integración de una partícula cargada en presencia de un dipolo magnético. Este notebook utiliza `DifferentialEquations` para resolver dicho problema. Juegue con las soluciones y la librería, y trate de entender cómo funciona. Ejecute distintas condiciones initiales, cambie las tolerancias, cambie los métodos, etc. La idea es únicamente que se familiarize con el paquete, no hace falta que presente nada respecto a este ejercicio.


*Nota: si quiere probar otros métodos además de Euler y Runge Kutta, puede consultar los métodos disponibles en el paquete en 
https://diffeq.sciml.ai/stable/solvers/ode_solve/*

*Puede consultar también https://diffeq.sciml.ai/stable/tutorials/ode_example/ para un pequeño tutorial del paquete.*

## Problema 2
**Estabilidad numérica**

***2a)***
Volvamos nuevamente la ecuación del notebook anterior

$$
\frac{dy}{dt} = \lambda y
$$

Resuelva la ecuación con $\lambda = -1$ en el intervalo temporal $[0,20]$, con $N = 100, 50, 30, 20, 10, 5$ (es decir, para $\Delta t$ creciente). Experimente nuevamente con Euler y Runge-Kutta y grafique. ¿Qué observa?

*Nota: utilice en la función `solve` los argumentos opcionales `saveat = dt` y `adaptive = false`.
Ejemplo:*
```julia
sol = solve(prob, RK4(), dt = dt, saveat = dt, adaptive = false)
```
*Puede averiguar más sobre estas opciones en https://diffeq.sciml.ai/stable/basics/common_solver_opts/ o preguntar en clases o en el aula virtual.*

***2b)*** Use el notebook *region_de_estabilidad* para investigar la región de convergencia de varios métodos de las librerías. Simplemente lea el notebook, entiéndalo, y corra los ejemplos, no hace falta que agregue nada. Pregunte en clase todas sus dudas.

## Problema 3
Construya su propio código para su ecuación favorita (alguna que no sea muy complicada, en la página del paquete DifferentialEquations de Julia hay varios ejemplos, o puede usar el dipolo 2D (orbitas coplanares)). Evolucione con distintas tolerancias, con paso fijo (vea como se hace esto en cada caso, use métodos fijos o adaptativos). Grafique diferencias de soluciones con distinta resolución o tolerancia y/o con distintos datos iniciales muy próximos.
Estudie convergencia, encuentre soluciones con distintos pasos (fijos), haga los cocientes y grafique el factor $Q$.

In [ ]:
#Función auxiliar para calcular el factor Q.
#Simplemente calcule tres soluciones con distintos dt
#Si sol_1 usa N+1 puntos, sol_2 debe usar 2N+1 y sol4 usar 4N+1.
#No olvide utilizar `adaptive = false` y `saveat = dt` 
using LinearAlgebra

function Q(t,sol_1,sol_2,sol_4)
    return norm(sol_1(t)[:] - sol_2(t)[1:2:end],2)/norm(sol_2(t)[1:2:end]-sol_4(t)[1:4:end],2)
end

In [ ]:
#Ejemplo simple
using Plots
λ = -im
T = (0.0,20.0)
N = 51
dt = (T[2]-T[1])/(N-1)
function CrecimientoExponencial!(du, u, par, t)
    λ = par
    du .= λ*u
end
y0 = [1.0+ 1.0im]
prob = ODEProblem(CrecimientoExponencial!,y0,T,λ);
#t = T[1]:dt:T[2]

sol_1 = solve(prob, RK4(), dt = dt  , saveat = dt,   adaptive = false);
sol_2 = solve(prob, RK4(), dt = dt/2, saveat = dt/2, adaptive = false);
sol_4 = solve(prob, RK4(), dt = dt/4, saveat = dt/4, adaptive = false);
plot(sol_1.t,map(t -> Q(t,sol_1,sol_2,sol_4),sol_1.t), yaxis=("Q",(0.,20)), label="Q")

## Problema 4
**Opcional para personas ambiciosas** 
Busque una ecuación que sea stiff y evolucione con un método apropiado. Un caso simple es la ecuación: 

$$
\begin{array}{ll}
u_t &= v \\
v_t &= \frac{1}{\epsilon}(u - v)
\end{array}
$$
con $\epsilon > 0$ muy pequeño. Este permite usar métodos IMEX.